In [6]:
import openai
print("✅ OpenAI version:", openai.__version__)


✅ OpenAI version: 1.55.3


In [7]:
import sys
print(sys.executable)


/usr/local/bin/python3


In [8]:
from dotenv import load_dotenv
import os

# Load .env
load_dotenv()

# Get key and ensure it's visible to the environment
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key  # 👈 ensures new SDK can auto-detect it

if api_key:
    print("✅ Key loaded and exported:", api_key[:12] + "..." + api_key[-5:])
else:
    raise ValueError("❌ No API key found — check your .env file")


✅ Key loaded and exported: sk-proj-dtvl...dV18A


In [9]:
from openai import OpenAI
client = OpenAI()  # now auto-detects the key
print("✅ Client initialized successfully.")


✅ Client initialized successfully.


In [14]:
# =========================================================
# 1️⃣ Imports & Setup
# =========================================================
from dotenv import load_dotenv
import os, pandas as pd, numpy as np
from tqdm import tqdm
from openai import OpenAI
import random, json
import time
time.sleep(25)   # wait 25 seconds between calls


# Load OpenAI Key from .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("✅ Key loaded:", api_key[:12] + "..." + api_key[-5:])
else:
    raise ValueError("❌ No API key found. Please add OPENAI_API_KEY to your .env")

client = OpenAI(api_key=api_key)

# =========================================================
# 2️⃣ Load Dataset
# =========================================================
df = pd.read_csv("/Users/pulinkumar/Desktop/ALGOVERSE/Financial-QA-10k.csv")
df = df.fillna("")
print(f"✅ Dataset loaded: {df.shape}")
print(df.head(3))

# Just in case columns differ
text_col = "context"
question_col = "question"
answer_col = "answer"

# =========================================================
# 3️⃣ Generate Model Predictions (Baseline LLMs)
# =========================================================
# You can replace these with actual calls to local models (Mistral / LLaMA)
# For now, we simulate random model predictions
def mock_model_prediction(question):
    endings = [
        "increased financial stability.",
        "improved risk-adjusted returns.",
        "lowered capital costs.",
        "helped diversification efforts."
    ]
    return f"The company's strategy led to {random.choice(endings)}"

df["mistral_pred"] = df[question_col].apply(lambda q: mock_model_prediction(q))
df["llama_pred"] = df[question_col].apply(lambda q: mock_model_prediction(q))

print("✅ Mock model predictions added for Mistral & LLaMA.")

# =========================================================
# 4️⃣ Define the LLM-as-a-Judge function
# =========================================================
def llm_as_judge(question, true_answer, pred_answer, judge_model="gpt-4o-mini"):
    """
    Use ChatGPT (GPT-5/4o-mini) to semantically evaluate model prediction quality.
    """
    prompt = f"""
You are a financial QA evaluation assistant.

Question: {question}

Ground Truth Answer: {true_answer}
Predicted Answer: {pred_answer}

Evaluate the predicted answer from 0.0 to 1.0 based on:
1. **Correctness** – factual alignment with the ground truth.
2. **Completeness** – inclusion of key details.
3. **Relevance** – focus on answering the question.

Return only valid JSON:
{{"score": 0.xx, "justification": "short reasoning"}}
"""
    try:
        response = client.chat.completions.create(
            model=judge_model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=200
        )
        content = response.choices[0].message.content.strip()
        result = json.loads(content)
        return result.get("score", 0.0), result.get("justification", "")
    except Exception as e:
        print("⚠️ Error:", e)
        return None, str(e)

# =========================================================
# 5️⃣ Evaluate Mistral & LLaMA Predictions
# =========================================================
results = []
judge_model = "gpt-4o-mini"  # or "gpt-5" if available

for model_name in ["mistral_pred", "llama_pred"]:
    print(f"\n🔎 Evaluating model: {model_name} with judge: {judge_model}")
    scores, reasons = [], []

    for _, row in tqdm(df.head(50).iterrows(), total=50):
        s, r = llm_as_judge(
            question=row[question_col],
            true_answer=row[answer_col],
            pred_answer=row[model_name],
            judge_model=judge_model
        )
        scores.append(s)
        reasons.append(r)

   # only attach to evaluated subset
    df_subset = df.head(len(scores)).copy()
    df_subset[f"{model_name}_score"] = scores
    df_subset[f"{model_name}_reason"] = reasons

    avg_score = np.mean([s for s in scores if s is not None])
    print(f"✅ {model_name} → Average Judge Score: {avg_score:.3f}")

    results.append({"model": model_name, "avg_score": avg_score, "judge": judge_model})

# =========================================================
# 6️⃣ Save Results
# =========================================================
results_df = pd.DataFrame(results)
results_df.to_csv("/Users/pulinkumar/Desktop/ALGOVERSE/Financial-Agent-LLM-Judge-Results.csv", index=False)
df.to_csv("/Users/pulinkumar/Desktop/ALGOVERSE/Financial-Agent-LLM-Detailed.csv", index=False)
print("\n💾 Results saved to:")
print("  • Financial-Agent-LLM-Judge-Results.csv")
print("  • Financial-Agent-LLM-Detailed.csv")


✅ Key loaded: sk-proj-dtvl...dV18A
✅ Dataset loaded: (7000, 5)
                                            question  \
0  What area did NVIDIA initially focus on before...   
1  What are some of the recent applications of GP...   
2  What significant invention did NVIDIA create i...   

                                              answer  \
0           NVIDIA initially focused on PC graphics.   
1  Recent applications of GPU-powered deep learni...   
2                   NVIDIA invented the GPU in 1999.   

                                             context ticker    filing  
0  Since our original focus on PC graphics, we ha...   NVDA  2023_10K  
1  Some of the most recent applications of GPU-po...   NVDA  2023_10K  
2  Our invention of the GPU in 1999 defined moder...   NVDA  2023_10K  
✅ Mock model predictions added for Mistral & LLaMA.

🔎 Evaluating model: mistral_pred with judge: gpt-4o-mini


  2%|▏         | 1/50 [00:01<01:33,  1.90s/it]

⚠️ Error: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-tlkrWf6nwuFRAyLrUMOXNnAn on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}


  4%|▍         | 2/50 [00:22<10:29, 13.11s/it]

⚠️ Error: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-tlkrWf6nwuFRAyLrUMOXNnAn on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}


...

 98%|█████████▊| 49/50 [1:07:55<00:27, 27.59s/it]

⚠️ Error: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-tlkrWf6nwuFRAyLrUMOXNnAn on tokens per min (TPM): Limit 100000, Used 99933, Requested 164. Please try again in 41m54.24s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


100%|██████████| 50/50 [1:08:17<00:00, 81.94s/it]

⚠️ Error: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-tlkrWf6nwuFRAyLrUMOXNnAn on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
✅ llama_pred → Average Judge Score: nan

💾 Results saved to:
  • Financial-Agent-LLM-Judge-Results.csv
  • Financial-Agent-LLM-Detailed.csv



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [11]:
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd, numpy as np, time, os
from tqdm import tqdm

# Load your key
load_dotenv("/Users/pulinkumar/Desktop/ALGOVERSE/Multi-Domain-Financial-Agent/.env")
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

print("✅ Key loaded:", api_key[:12] + "..." + api_key[-5:])


✅ Key loaded: sk-proj-dtvl...dV18A


In [12]:
# Load your Financial QA dataset
df = pd.read_csv("/Users/pulinkumar/Desktop/ALGOVERSE/Financial-QA-10k.csv")
print("✅ Dataset loaded:", df.shape)
print(df.head(3))

# Mock predictions for models (example)
df["mistral_pred"] = df["answer"].sample(frac=1, random_state=42).values
df["llama_pred"] = df["answer"].sample(frac=1, random_state=24).values
print("✅ Mock model predictions added for Mistral & LLaMA.")


✅ Dataset loaded: (7000, 5)
                                            question  \
0  What area did NVIDIA initially focus on before...   
1  What are some of the recent applications of GP...   
2  What significant invention did NVIDIA create i...   

                                              answer  \
0           NVIDIA initially focused on PC graphics.   
1  Recent applications of GPU-powered deep learni...   
2                   NVIDIA invented the GPU in 1999.   

                                             context ticker    filing  
0  Since our original focus on PC graphics, we ha...   NVDA  2023_10K  
1  Some of the most recent applications of GPU-po...   NVDA  2023_10K  
2  Our invention of the GPU in 1999 defined moder...   NVDA  2023_10K  
✅ Mock model predictions added for Mistral & LLaMA.


In [13]:
def ask_judge(question, reference, prediction, model="gpt-4o-mini"):
    """Compare model answer vs. ground truth using an LLM judge."""
    prompt = f"""
    You are an expert evaluator.
    Compare the following:

    ❓ Question: {question}
    ✅ True Answer: {reference}
    🤖 Model Answer: {prediction}

    Rate the similarity on a 0–1 scale (0 = completely wrong, 1 = perfect),
    and briefly explain your reasoning.
    Return ONLY JSON in this format:
    {{ "score": 0.x, "reason": "..." }}
    """

    for attempt in range(5):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
            )
            text = response.choices[0].message.content
            import json
            j = json.loads(text)
            return j.get("score", None), j.get("reason", "")
        except Exception as e:
            err = str(e)
            if "429" in err:
                wait = 25
                print(f"⚠️ Rate limit hit. Waiting {wait}s...")
                time.sleep(wait)
                continue
            else:
                print("⚠️ Error:", e)
                return None, ""
    return None, ""
